# <span style="color:red"> Lecture 15 - Grouping, Aggregating, & Merging Data </span>

<font size = "4">

In the previous class we covered

- Missing values
- The basics of data cleaning

This class we will talk about 
- Extracting groups of a dataset
- Computing aggregate statistics by group
- Merging datasets

# <span style="color:red"> I. Grouping </span>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size = "4">
We will use the Formula One datasets, stored in the folder "f1_data_raw". 
<br>

[See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)
<br>

We'll begin with "results.csv". Using ``.dtypes`` allows us to see all the column headings, along with their data-types


In [ ]:
df_results = pd.read_csv("f1_data_raw/results.csv")
print(df_results.dtypes)

<font size = "4">

There are 5 "ID" columns, resultId, raceId, driverId, constructorId, statusId.


Let's check how how many rows there are in the dataset, and how many **unique** values of these ID's there are

In [ ]:
# We've used .shape in the past. 
# But the "len" function works too
# The last two show other alternatives
print("Number of rows:", len(df_results))
print("Unique resultId:", len(pd.unique(df_results["resultId"])))
print("Unique raceId:", len(pd.unique(df_results["raceId"])))
print("Unique driverId:", len(pd.unique(df_results["driverId"])))
print("Unique constructorId:", len(df_results["constructorId"].unique()))
print("Unique statusId:", df_results["statusId"].nunique())


<font size = "4">

Based on a driver's finish, a certain number of points is assigned, which corresponds to the "points" column.


We could, for example, compute the mean of this column to compute the average number of points per result. This doesn't seem like a very interesting statistic however.

But, if we could compute the average points for each driver, or each constructor (the entity which built the car). This would give us important information: which drivers/cars perform the best.

We can do this in Pandas using the ``.groupby`` method

In [ ]:
driver_group = df_results.groupby(by = "driverId")
constructor_group = df_results.groupby(by = "constructorId")

# First question to always ask with a new function:
# what kind of thing does it output?
print(type(driver_group))

# 2nd: what does it look like?
print(driver_group)

<font size = "4">

This returns a "DataFrameGroupBy" object, and printing it gives no information (just like for ``range`` and ``zip``).

When you do ``range(10)`` it does **not** create an object with the numbers 0, 1, 2, ..., 8, 9. It creates a range object that "knows" how to generate the integers from 0 to 9 one-by-one. It doesn't generate these numbers until they are needed, such as a for loop.

When you do ``zip(list_1, list_2)`` it does **not** create an object containing all the pairs between the lists. It creates a zip object that "knows" how to generate the pairs one-by-one. It doesn't generate these pairs until they are needed, such as a for loop. 



In [ ]:
list_1 = [5.3, "Pandas", -9]
list_2 = ["Number", 0, 12]

zip_lists = zip(list_1, list_2)


print(type(zip_lists))
print(zip_lists)

# The variable zip_lists is "associated" or "linked" 
# with the original lists. It draws the elements from 
# those lists when needed.

<font size = "4">

Similarly, a DataFrameGroupBy object is "associated" or "linked" with the original DataFrame object. To get something useful, we need to specify:

- Column/variable
- An operation

In [ ]:
# Choose "points" and compute its average
driver_mean_points = driver_group["points"].mean()
print(driver_mean_points)
print()
print(type(driver_mean_points))

<font size = "4">

We can see that "Driver 1" scores an average of 14.31 points per race. Driver 2 only scores an average of 1.41 points per race.

Let's grab the 5 top performing drivers

In [ ]:
mean_pts_sorted = driver_mean_points.sort_values(ascending = False)

# This is a *Series*. Only needs one integer (or range of integers).
# A DataFrame needs two integers (row and column)
print(mean_pts_sorted.iloc[:5]) 

<font size = "4">

To go from driverId to the driver's name, you would need to use "drivers.csv". The best performing driver (with driverId 1) is Lewis Hamilton
<br>

Could also compute the mean of both "points" and "positionOrder" (since they are both numerical)

In [ ]:
mean_pts_position = driver_group[["points", "positionOrder"]].mean()
print(mean_pts_position)

<font size = "4">

Can "chain" together all the commands instead of breaking them up into single lines

In [ ]:
#### Starting with the DataFrame, we did the following steps to get
#### the top 5 performers:

# driver_group = df_results.groupby(by = "driverId")
# driver_mean_points = driver_group["points"].mean()
# mean_pts_sorted = driver_mean_points.sort_values(ascending = False)
# print(mean_pts_sorted.iloc[:5])


top5_one_line = df_results.groupby(by = "driverId")["points"].mean().sort_values(ascending=False).iloc[:5]
print(top5_one_line)

In [ ]:
# It's considered good practice to make each line less than 80 characters
# This makes it easier to scroll up and down without going sideways.
# Can use the backslash (\) for a new line

top5_one_line = df_results.groupby(by = "driverId")["points"].mean().\
    sort_values(ascending=False).iloc[:5]
print(top5_one_line)

In [ ]:
# Can also add parentheses around everything on the right
# Then can use multiple lines

top5_one_line = (df_results.groupby(by = "driverId")["points"].mean().
    sort_values(ascending=False).iloc[:5])
print(top5_one_line)

# <span style="color:red"> II. Aggregating Statistics </span>

<font size = "4">

Suppose we want to compute the mean, standard deviation, minimum, and maximum of "points" for the entire dataset

Most straightforward way is:

In [ ]:
pts_col = df_results["points"]
pts_mean = pts_col.mean()
pts_std = pts_col.std()
pts_min = pts_col.min()
pts_max = pts_col.max()

<font size = "4">

Can compute these in one command using ``.agg`` (which stands for aggregate)

In [ ]:
pts_col = df_results["points"]

pts_stats = pts_col.agg(["mean", "std", "min", "max"])

print(pts_stats)
print()
print(type(pts_stats))

<font size = "4">

The ``.agg`` function is very flexible. By rule, the more flexible a function is, the harder it is to learn. 

Both Pandas Series and DataFrames have a method called `.agg`. If you type ``help(df_results.agg)``, you will see the documentation, along with some very simple examples.

Note that the examples for ``help(pts_col.agg)`` will be different, because it is a **Series**, while "df_results" is a **DataFrame**

In [ ]:
help(df_results.agg)

<font size = "4">

Let's look at the examples for a DataFrame:

In [ ]:
df_example = pd.DataFrame([[1, 2, 3],
                           [4, 5, 6],
                           [7, 8, 9], 
                           [np.nan, np.nan, np.nan]], 
                           columns=['A', 'B', 'C'])
print(df_example)

In [ ]:
# Aggregate the sum and minimum functions over the rows
example_1 = df_example.agg(["sum", "min"])
print(example_1)

In [ ]:
# Different aggregations per column
example_2 = df_example.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
print(example_2)

In [ ]:
# Aggregate different functions over the columns 
# and rename the index of the resulting DataFrame.
example_3 = df_example.agg(x = ('A', 'max'),
                           y = ('B', 'min'),
                           z = ('C', 'mean'))

print(example_3)

In [ ]:
# Aggregate over the columns
example_4 = df_example.agg("mean", axis="columns")
print(example_4)

<font size = "4">

With these examples in mind, we can return to our original DataFrame

In [ ]:
results_agg = df_results.agg(mean_points = ('points','mean'),
                          sd_points =   ('points','std'),
                          min_points =  ('points','min'),
                          max_points =  ('points','max'))

print(results_agg)
results_agg

In [ ]:
count_unique = lambda col: len(col.unique())
results_agg = df_results.agg(mean_points = ('points','mean'),
                          mean_laps =   ('laps','mean'),
                          min_points =  ('points','min'),
                          max_points =  ('points','max'),
                          num_drivers = ('driverId', count_unique))

print(results_agg)
results_agg

# <span style="color:red"> III. Grouping + Aggregating </span>


<img src="figures/agg.png" alt="drawing" width="400"/>

In [ ]:
drivers_agg = (df_results.groupby("driverId")
                      .agg(mean_points = ('points','mean'),
                           sd_points =   ('points','std'),
                           min_points =  ('points','min'),
                           max_points =  ('points','max'),
                           appearances   = ('points',len)))

print(type(drivers_agg))
print(drivers_agg)

In [ ]:
# driverId is the "Index" column, NOT a regular column
print(drivers_agg.columns.values)

<font size = "4" >

Groupby + Aggregate statistics (multigroup)

Each constructor can have multiple vehicles competing in a given race. We'll group by Race ID and Constructor ID, then aggregate statistics. This will allow us to see how each constructor did relative to the others in a given race.

In [ ]:
teamrace_agg = (df_results.groupby(  ["raceId","constructorId"]    )
                       .agg(mean_points = ('points','mean'),
                            sd_points =   ('points','std'),
                            min_points =  ('points','min'),
                            max_points =  ('points','max'),
                            cars_entered   = ('points',len)))

print(teamrace_agg)

<font size = "4">

Filtering + Grouping + Aggregating: <br>

```python 
.query().groupby().agg()
```

- Another example of "chaining"

In [ ]:
# The following gets a subset of the data using .query()
# In this case we subset the data before computing aggregate statistics
# Note: "filtering" is often the word used to obtain a subset

teamrace_agg = (df_results.query("raceId >= 500")
                       .groupby(["raceId","constructorId"])
                        .agg(mean_points = ('points','mean'),
                             sd_points =   ('points','std'),
                             min_points =  ('points','min'),
                             max_points =  ('points','max'),
                             cars_entered   = ('points',len)))
teamrace_agg

In [ ]:
# maybe we're only interested in the mean
(df_results.query("raceId >= 500").groupby(["raceId","constructorId"]).
 agg(mean_points = ('points','mean')))

<font size = "4">

**Exercise:** Perform the following by chaining. Create a DataFrame where for each race (identified by "raceId") we aggregate the average number of laps and the average number of points.

In [ ]:
# your code here


<font size = "4">

**Exercise:** Perform the following by chaining. For each constructor (identified by "constructorId"), aggregate the average number of points, then sort in descending order.

In [ ]:
# your code here



# <span style="color:red"> IV. Merging </span>


<img src="figures/merge_stats.png" alt="drawing" width="600"/>

<font size = "4">

We have the original DataFrame ("df_results"), and we have aggregate statistics for each driver ("drivers_agg"). We'll merge these two together

In [ ]:
df_results

In [ ]:
drivers_agg

In [ ]:
# This command merges the "aggregate" information in "driver_agg" into
# "df_results" similar as the figure above.
# The merging variable "on" is determined by "driverId", which is a column
# that is common to both DataFrames
# "how = left" indicates that the left DataFrame is the baseline

results_merge = pd.merge(df_results,
                         drivers_agg,
                         on = "driverId",
                         how = "left")

In [ ]:
results_merge

<font size = "4">

**Exercise:** Compute a scatter plot of "points" (horizontal axis) vs. "mean_points" (vertical axis). This plot tries to describe how much a driver's performance in individual races deviates from their overall average.

In [ ]:
# your code here



<font size = "4">

**Exercise:** Merge the "teamrace_agg" data into "df_results". This time use the option:

```python
        on = ["raceId","constructorId"]
```

In [ ]:
# your code here


